In [16]:
# print out correlation and rmse results for predicted P(T) and predicted ratings for semisynthetc data for full model, reports only model, and ratings only model
import os
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
import warnings
plt.rcParams['font.family'] = 'serif'

# Set the font used for math expressions to LaTeX
plt.rcParams["mathtext.fontset"] = "cm"

In [2]:
# file paths
base_file = '/share/garg/311_data/sb2377/clean_codebase/three_year_base.csv'
type_rating_observed_base_file = '/share/garg/311_data/sb2377/clean_codebase/three_year_type_rating_observed_base.csv'
results_dir = '/share/garg/311_data/sb2377/results'

# user specified arguments
types = {'Street': 'StreetConditionDOT',
         'Park': 'MaintenanceorFacilityDPR',
         'Rodent': 'RodentDOHMH',
         'Food': 'FoodDOHMH',
         'DCWP': 'ConsumerComplaintDCWP'}
models = {'Full model': {'job_ids':[i * 3 + 3200 for i in range(20)]},
          'Ratings-only model': {'job_ids':[i * 3 + 3202 for i in range(20)]},
          'Reports-only model': {'job_ids':[i * 3 + 3201 for i in range(20)]}}
epoch = '59'

# plotting parameters
print_p_values = False

In [3]:
# load files
base_df = pd.read_csv(base_file)
type_rating_observed_base_df = pd.read_csv(type_rating_observed_base_file)

In [4]:
# get type indices
# for df with all types
type_df = base_df[['typeagency', 'type_idxs']].drop_duplicates()
indices = {}
for type_name, type_id in types.items():
    idx = type_df[type_df['typeagency'] == type_id]['type_idxs'].iloc[0]
    indices[type_name] = idx

# for df with only types with observed ratings
type_df = type_rating_observed_base_df[['typeagency', 'type_idxs']].drop_duplicates()
type_rating_observed_indices = {}
for type_name, type_id in types.items():
    idx = type_df[type_df['typeagency'] == type_id]['type_idxs'].iloc[0]
    type_rating_observed_indices[type_name] = idx

In [5]:
# get predicted ratings for all jobs for types with observed ratings
checkpoint_file = '{}/job{}/model-epoch={}.ckpt'
results_file = '{}/job{}/epoch={}_test.pkl'
checkpoint_counters = {}
results_counters = {}
for m in models:
    checkpoint_counters[m] = 0
    results_counters[m] = 0
type_rating_observed_dfs = {}
for m in models:
    type_rating_observed_dfs[m] = []

for m in models:
    for i, job_idx in enumerate(models[m]['job_ids']):
        if os.path.exists(checkpoint_file.format(results_dir, job_idx, epoch)):
            checkpoint_counters[m] += 1
        if os.path.exists(results_file.format(results_dir, job_idx, epoch)):
            results_counters[m] += 1
            with open(results_file.format(results_dir, job_idx, epoch), 'rb') as file:
                pred_rating, true_rating, mask, node_embedding, type_embedding, node_idxs, type_idxs, demographics, pred_pt, true_t = pickle.load(file)

            df = pd.DataFrame()
            df['pred_rating'] = pred_rating
            df['true_rating'] = true_rating
            df['node_idxs'] = node_idxs
            df['type_idxs'] = type_idxs
            df['pred_pt'] = pred_pt
            df['true_t'] = true_t
            df['mask'] = mask

            type_rating_observed_dfs[m].append(df)

for m in models:
    print('{}: checkpoint files done = {}'.format(m, checkpoint_counters[m]))
    print('{}: results files done = {}'.format(m, results_counters[m]))

Full model: checkpoint files done = 20
Full model: results files done = 20
Ratings-only model: checkpoint files done = 20
Ratings-only model: results files done = 20
Reports-only model: checkpoint files done = 20
Reports-only model: results files done = 20


In [6]:
# get predicted ratings for all jobs for types with unobserved ratings
checkpoint_file = '{}/job{}/model-epoch={}.ckpt'
results_file = '{}/job{}/epoch={}_test_unobserved.pkl'
checkpoint_counters = {}
results_counters = {}
type_rating_unobserved_models = ['Full model', 'Reports-only model']
for m in type_rating_unobserved_models:
    checkpoint_counters[m] = 0
    results_counters[m] = 0
type_rating_unobserved_dfs = {}
for m in type_rating_unobserved_models:
    type_rating_unobserved_dfs[m] = []

for m in type_rating_unobserved_models:
    for i, job_idx in enumerate(models[m]['job_ids']):
        if os.path.exists(checkpoint_file.format(results_dir, job_idx, epoch)):
            checkpoint_counters[m] += 1
        if os.path.exists(results_file.format(results_dir, job_idx, epoch)):
            results_counters[m] += 1
            with open(results_file.format(results_dir, job_idx, epoch), 'rb') as file:
                pred_rating, true_rating, mask, node_embedding, type_embedding, node_idxs, type_idxs, demographics, pred_pt, true_t = pickle.load(file)

            df = pd.DataFrame()
            df['pred_rating'] = pred_rating
            df['true_rating'] = true_rating
            df['node_idxs'] = node_idxs
            df['type_idxs'] = type_idxs
            df['pred_pt'] = pred_pt
            df['true_t'] = true_t
            df['mask'] = mask

            type_rating_unobserved_dfs[m].append(df)

for m in type_rating_unobserved_models:
    print('{}: checkpoint files done = {}'.format(m, checkpoint_counters[m]))
    print('{}: results files done = {}'.format(m, results_counters[m]))

Full model: checkpoint files done = 20
Full model: results files done = 20
Reports-only model: checkpoint files done = 20
Reports-only model: results files done = 20


In [7]:
# combine predicted P(T) and ratings for types with observed ratings and types with unobserved ratings
dfs = {}
for m in type_rating_unobserved_models:
    dfs[m] = []
    for i in range(len(type_rating_observed_dfs[m])):
        full_df = pd.concat([type_rating_observed_dfs[m][i], type_rating_unobserved_dfs[m][i]])
        dfs[m].append(full_df)
dfs['Ratings-only model'] = type_rating_observed_dfs['Ratings-only model']

In [17]:
# print out correlation and rmse results for predicted P(T)
for m in type_rating_unobserved_models:
    df_set = dfs[m]
    corrs = []
    rmses = []
    for idx in range(len(dfs[m][0]['type_idxs'].unique())):
        type_corrs = []
        type_rmses = []
        for df in df_set:
            df_type = df[df['type_idxs'] == idx]
            node_df = df_type.groupby(['node_idxs', 'type_idxs']).mean().reset_index()

            # calculate correlation
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                corr = pearsonr(node_df['pred_pt'], node_df['true_t'])
            type_corrs.append(corr[0])
            if print_p_values and corr[1] > 0.001:
                print(m, idx, corr)

            # calculate rmse
            rmse = np.sqrt(mean_squared_error(node_df['pred_pt'], node_df['true_t']))
            type_rmses.append(rmse)

        corrs.append(type_corrs)
        rmses.append(type_rmses)
    
    with warnings.catch_warnings():
        # calculate mean and 95% confidence interval over correlations
        warnings.simplefilter("ignore")
        corrs = np.array(corrs)
        filtered_corrs = corrs[~np.isnan(corrs).any(axis=1)]
        mean_for_each_job = filtered_corrs.mean(axis=0)
        mean_overall = filtered_corrs.mean()
        se_across_jobs = np.std(mean_for_each_job) / np.sqrt(len(mean_for_each_job) - 1)
        print('Model: {}, Corr: {:.4f} \pm {:.4f}'.format(m, mean_overall, 1.96 * se_across_jobs))

        # calculate mean and 95% confidence interval over rmses
        rmses = np.array(rmses)
        filtered_rmses = rmses[~np.isnan(rmses).any(axis=1)]
        mean_for_each_job = filtered_rmses.mean(axis=0)
        mean_overall = filtered_rmses.mean()
        se_across_jobs = np.std(mean_for_each_job) / np.sqrt(len(mean_for_each_job) - 1)
        print('Model: {}, RMSE: {:.4f} \pm {:.4f}'.format(m, mean_overall, 1.96 * se_across_jobs))


Model: Full model, Corr: 0.4680 \pm 0.0020
Model: Full model, RMSE: 0.0774 \pm 0.0009
Model: Reports-only model, Corr: 0.4840 \pm 0.0007
Model: Reports-only model, RMSE: 0.0586 \pm 0.0001


In [20]:
# print out correlation and rmse results for predicted ratings
for m in models:
    df_set = type_rating_observed_dfs[m]
    corrs = []
    rmses = []
    for t in types:
        type_corrs = []
        type_rmses = []
        idx = indices[t]
        type_rating_observed_idx = type_rating_observed_indices[t]
        for df in df_set:
            df_type = df[df['type_idxs'] == idx]
            if m == 'Ratings-only model':
                df_type = df[df['type_idxs'] == type_rating_observed_idx]
            else:
                df_type = df[df['type_idxs'] == idx]
            node_df = df_type.groupby(['node_idxs', 'type_idxs']).mean().reset_index()

            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                if m == 'Reports-only model':
                    # for reports-only model, we use -P(T) as a proxy for r
                    corr = pearsonr(-1 * node_df['pred_pt'], node_df['true_rating'])
                else:
                    corr = pearsonr(node_df['pred_rating'], node_df['true_rating'])
            type_corrs.append(corr[0])
            if print_p_values and corr[1] > 0.001:
                print(m, idx, corr)

            # calculate rmse
            if m == 'Reports-only model':
                rmse = np.nan
            else:
                rmse = np.sqrt(mean_squared_error(node_df['pred_rating'], node_df['true_rating']))
            type_rmses.append(rmse)

        corrs.append(type_corrs)
        rmses.append(type_rmses)
    
    with warnings.catch_warnings():
        # calculate mean and 95% confidence interval over correlations
        warnings.simplefilter("ignore")
        corrs = np.array(corrs)
        filtered_corrs = corrs[~np.isnan(corrs).any(axis=1)]
        mean_for_each_job = filtered_corrs.mean(axis=0)
        mean_overall = filtered_corrs.mean()
        se_across_jobs = np.std(mean_for_each_job) / np.sqrt(len(mean_for_each_job) - 1)
        print('Model: {}, Corr: {:.4f} \pm {:.4f}'.format(m, mean_overall, 1.96 * se_across_jobs))

        # calculate mean and 95% confidence interval over rmses
        rmses = np.array(rmses)
        filtered_rmses = rmses[~np.isnan(rmses).any(axis=1)]
        mean_for_each_job = filtered_rmses.mean(axis=0)
        mean_overall = filtered_rmses.mean()
        se_across_jobs = np.std(mean_for_each_job) / np.sqrt(len(mean_for_each_job) - 1)
        print('Model: {}, RMSE: {:.4f} \pm {:.4f}'.format(m, mean_overall, 1.96 * se_across_jobs))


Model: Full model, Corr: 0.6243 \pm 0.0075
Model: Full model, RMSE: 1.0358 \pm 0.0257
Model: Ratings-only model, Corr: 0.6246 \pm 0.0075
Model: Ratings-only model, RMSE: 1.0331 \pm 0.0262
Model: Reports-only model, Corr: 0.3345 \pm 0.0077
Model: Reports-only model, RMSE: nan \pm nan
